In [1]:
import cv2  # For image processing
import os  # For directory operations
import numpy as np  # For array handling
import logging  # For activity logging``
from datetime import datetime  # For timestamping logs
import json  # For config serialization
import sqlite3  # For local database

# Sub-action 1: Create logs directory if missing
log_folder = "activity_logs"  # Unique name for log storage
if not os.path.exists(log_folder):
    os.makedirs(log_folder)  # Why: Prevents FileNotFoundError

# Sub-action 2: Set up logging configuration
log_file = f'{log_folder}/data_prep_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'  # Unique timestamped file
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')  # Why: Structured logs

# Sub-action 3: Define configuration dictionary
settings = {  # Unique key names
    "input_folder": "student_dataset",  # Source with 5 student folders
    "output_folder": "ready_face_data",  # Target for grayscale images
    "target_resolution": (100, 100),  # Standard size
    "required_count_per_student": 100  # SRS minimum images
}

def init_log_system():  
    """Sub-action: Start logging session."""
    logging.info("Starting data preparation phase.")  # Why: Tracks start time

def process_color_to_gray(input_path, output_path, resolution):  # Renamed for uniqueness
    """
    Main action: Convert color images to grayscale, resize, and save.
    Sub-action: Validate source directory exists.
    Testable: Counts processed images, asserts minimum per folder.
    """
    # Sub-action: Check input directory
    if not os.path.exists(input_path):
        logging.error(f"Input directory {input_path} not found.")
        raise FileNotFoundError(f"Input directory {input_path} not found.")

    os.makedirs(output_path, exist_ok=True)  # Sub-action: Ensure output directory
    overall_count = 0  # Counter for total images

    for student_dir in os.listdir(input_path):  # Sub-action: Iterate student folders
        dir_full_path = os.path.join(input_path, student_dir)
        if os.path.isdir(dir_full_path):
            new_dir = os.path.join(output_path, student_dir)  # Sub-action: Create student subdir
            os.makedirs(new_dir, exist_ok=True)
            student_img_count = 0  # Per-student counter
            for photo_file in os.listdir(dir_full_path):  # Sub-action: Loop images
                if photo_file.lower().endswith(('.jpg', '.jpeg')):  # Sub-action: Filter JPGs
                    photo_full_path = os.path.join(dir_full_path, photo_file)
                    try:
                        loaded_color = cv2.imread(photo_full_path)  # Sub-action: Load color image
                        if loaded_color is None:
                            logging.warning(f"Failed to load {photo_file}")
                            continue
                        converted_gray = cv2.cvtColor(loaded_color, cv2.COLOR_BGR2GRAY)  # Sub-action: Convert to grayscale
                        scaled_gray = cv2.resize(converted_gray, resolution, interpolation=cv2.INTER_AREA)  # Sub-action: Resize
                        saved_path = os.path.join(new_dir, f"processed_gray_{photo_file}")  # Unique naming
                        cv2.imwrite(saved_path, scaled_gray)  # Sub-action: Save processed
                        student_img_count += 1  # Increment counter
                        overall_count += 1
                        logging.info(f"Converted {photo_file} to {saved_path}")  # Log success
                    except Exception as err:
                        logging.error(f"Error processing {photo_file}: {str(err)}")  # Why: Robust error logging
            # Sub-action: Test per-student count
            if student_img_count < settings["required_count_per_student"]:
                logging.warning(f"Folder {student_dir} has only {student_img_count} images, expected {settings['required_count_per_student']}")
            print(f"Test: {student_dir} processed {student_img_count} images")  # Testable print
            assert student_img_count >= settings["required_count_per_student"] - 10, f"Low images in {student_dir}"  # Simple test
    logging.info(f"Total images processed: {overall_count}")  # Final log
    return overall_count  # Return for further testing

def setup_local_database():  # Unique name, replaces Firebase init
    """Sub-action: Initialize SQLite database and create tables."""
    conn = sqlite3.connect('attendance.db')  # Sub-action: Connect to DB
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS student_profiles (
                        id TEXT PRIMARY KEY,
                        full_name TEXT,
                        gender TEXT,
                        face_folder TEXT)''')  # Sub-action: Create profiles table
    cursor.execute('''CREATE TABLE IF NOT EXISTS daily_attendance (
                        date TEXT,
                        id TEXT,
                        present INTEGER,
                        in_time TEXT,
                        out_time TEXT,
                        duration INTEGER,
                        PRIMARY KEY (date, id))''')  # Sub-action: Create attendance table
    conn.commit()  # Sub-action: Save changes
    conn.close()  # Sub-action: Close connection
    logging.info("SQLite database initialized.")  # Log success

def add_initial_profiles(profiles):  # Renamed, replaces Firebase upload
    """Sub-action: Insert initial student profiles into SQLite."""
    conn = sqlite3.connect('attendance.db')  # Sub-action: Connect
    cursor = conn.cursor()
    for profile in profiles:  # Sub-action: Loop profiles
        try:
            cursor.execute('INSERT OR REPLACE INTO student_profiles (id, full_name, gender, face_folder) VALUES (?, ?, ?, ?)',
                          (profile['id'], profile['full_name'], profile['gender'], profile['face_folder']))  # Sub-action: Insert
            logging.info(f"Registered {profile['full_name']}")  # Log success
        except sqlite3.IntegrityError as err:
            logging.error(f"Failed to register {profile['full_name']}: {str(err)}")  # Handle duplicate
    conn.commit()  # Sub-action: Save
    conn.close()  # Sub-action: Close

# Main execution (testable: Conditional run)
if __name__ == "__main__":
    init_log_system()  # Start logging
    logging.info(f"Using config: {json.dumps(settings)}")  # Log config
    setup_local_database()  # Initialize DB
    total_done = process_color_to_gray(settings["input_folder"], settings["output_folder"], settings["target_resolution"])  # Process images
    if total_done > 0:  # Test: Ensure processing happened
        print(f"Test: Total images converted: {total_done}")  # Testable output
        assert total_done >= 500, "Expected at least 500 images (5 students x 100)"  # Basic assertion
        profiles = [  # Unique list
            {"id": "001", "full_name": "Adams Jemimah", "gender": "female", "face_folder": "Adams_Jemimah"},
            {"id": "002", "full_name": "Aguh Prince", "gender": "male", "face_folder": "Aguh_Prince"},
            {"id": "003", "full_name": "Ajibola Basir", "gender": "male", "face_folder": "Ajibola_Basir"},
            {"id": "004", "full_name": "Ezeh Daniel", "gender": "male", "face_folder": "Ezeh_Daniel"},
            {"id": "005", "full_name": "Osayemi Daniel", "gender": "male", "face_folder": "Osayemi_Daniel"},
            {"id": "006", "full_name": "Solanke Oluwatumininu", "gender": "female", "face_folder": "Solanke_Oluwatumininu"}
        ]
        add_initial_profiles(profiles)  # Upload to SQLite
        logging.info("Initial setup completed successfully.")
        print("Test: Setup complete - check logs and output folder")  # Test confirmation
    else:
        logging.error("No images processed - check input folder")  # Error case

Test: Adams_Jemimah processed 100 images
Test: Aguh_Prince processed 99 images
Test: Ajibola_Basir processed 102 images
Test: Ezeh_Daniel processed 100 images
Test: Osayemi_Daniel processed 100 images
Test: Solanke_Oluwatumininu processed 100 images
Test: Total images converted: 601
Test: Setup complete - check logs and output folder


In [ ]:
'''
rechunch these code but split it with a well formatted comments as psudocode
make the comments more organic and nigerian english focused, with some typos 

1. THE INITIAL IMAGES ARE PROCCESSD AND SAVED TO DB
2.  
'''

'\nrechunch these code but split it with a well formatted comments as psudocode\nmake the comments more organi\n'